This project will use RAG (Retrieval Augmented Generation) to ensure our question/answering assistant has high accuracy.

In [1]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr

c:\Users\natha\Desktop\FACENS\TCC\ai-llm\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# imports for langchain

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter

In [3]:
from typing import List

def load_documents(base_path: str, documents: List, text_loader_kwargs: dict = None):
    """
    Carrega arquivos .md de um caminho base. Usa DirectoryLoader para pastas com subdiretórios
    e TextLoader para pastas com apenas arquivos .md.

    Args:
        base_path (str): Caminho onde estão os arquivos ou subpastas.
        documents (List): Lista onde os documentos serão adicionados.
        text_loader_kwargs (dict, optional): Opções para o TextLoader.
    """
    if text_loader_kwargs is None:
        text_loader_kwargs = {'encoding': 'utf-8'}

    # Verifica se há subdiretórios
    subfolders = [f for f in glob.glob(os.path.join(base_path, "*")) if os.path.isdir(f)]

    if subfolders:
        # Caso tenha subpastas, usar DirectoryLoader para cada uma
        for folder in subfolders:
            doc_type = os.path.basename(folder)
            print(f"[DirectoryLoader] Carregando de: {doc_type}")
            loader = DirectoryLoader(
                folder,
                glob="**/*.md",
                loader_cls=TextLoader,
                loader_kwargs=text_loader_kwargs
            )
            folder_docs = loader.load()
            for doc in folder_docs:
                doc.metadata["doc_type"] = doc_type
                documents.append(doc)
    else:
        # Caso não tenha subpastas, assumimos que a pasta contém arquivos .md diretamente
        md_files = glob.glob(os.path.join(base_path, "*.md"))
        for file_path in md_files:
            print(f"[TextLoader] Carregando arquivo: {os.path.basename(file_path)}")
            loader = TextLoader(file_path, **text_loader_kwargs)
            doc = loader.load()[0]
            doc.metadata["doc_type"] = os.path.basename(base_path)
            documents.append(doc)


documents = []

load_documents("../preprocessing/knowledge-base", documents)

print(f"Total de documentos carregados: {len(documents)}")      
print(documents)  

[DirectoryLoader] Carregando de: company
[DirectoryLoader] Carregando de: products
Total de documentos carregados: 13
[Document(metadata={'source': '..\\preprocessing\\knowledge-base\\company\\Estratégia de Expansão - GPT.md', 'doc_type': 'company'}, page_content='# Expansão da Operação e Aumento do Faturamento\n\n## 1. Diagnóstico Inicial\n\n- Atuação no Brasil com foco em mandriladoras portáteis para reparo em campo.\n- Leads gerados por Google Ads, indicações e repasses do fabricante.\n- Vendas apenas para leads inbound, sem cold call.\n- Concorrência com produtos mais baratos (menor qualidade) ou mais caros.\n- Nichos: mineradoras e construção civil.\n- Oferece suporte pós-venda e peças de reposição.\n- Não realiza aluguel nem financiamento próprio (somente via bancos).\n\n## 2. Estruturação Interna\n\n- **Playbook de Vendas**: Padroniza atendimento e melhora conversão.\n- **Bot WhatsApp**: Automatiza triagem e qualificação inicial.\n- **CRM (RD Station)**: Gerencia leads, etapas e

In [4]:
documents[10]

Document(metadata={'source': '..\\preprocessing\\knowledge-base\\products\\S50\\Relação de Itens - S50.md', 'doc_type': 'products'}, page_content='## Lista de Itens S50\n\n| Imagem | Item                                   | Qtd.       | Imagem | Item                                              | Qtd. |\n| ------ | -------------------------------------- | ---------- | ------ | ------------------------------------------------- | ---- |\n|        | Motor c/ Caixa Redutora                | 1          |        | Painel de controle                                | 1    |\n|        | Controle Remoto p/ Usinagem e Soldagem | 2          |        | Fuso de avanço                                    | 1    |\n|        | Braço de fixação (Ferro)               | 6          |        | Braço de fixação (Aluminio)                       | 6    |\n|        | Kit parafusos de fixação dos braços    | 2 (12 un.) |        | Cone de centralização da barra de 50mm            | 2    |\n|        | Cone de centr

In [5]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

Created a chunk of size 1721, which is longer than the specified 1000
Created a chunk of size 1196, which is longer than the specified 1000


In [6]:
len(chunks)

48

In [7]:
chunks[5]

Document(metadata={'source': '..\\preprocessing\\knowledge-base\\company\\Pesquisa de Mercado Mandrilhadora.md', 'doc_type': 'company'}, page_content='### Setores que utilizam mandrilhadoras portáteis:\n\n- **Manutenção Industrial:** usinas, siderúrgicas, indústrias químicas\n- **Óleo e Gás:** plataformas, tubulações, refino\n- **Energia:** turbinas, geradores, usinas térmicas/hidrelétricas\n- **Mineração:** máquinas pesadas, britagem\n- **Naval:** navios, plataformas, estaleiros\n- **Transporte:** manutenção de vagões, locomotivas, ferrovias\n\n---\n\n## 2. Pesquisa de Campo\n\nO mercado brasileiro apresenta forte presença de empresas de manutenção que fabricam ou comercializam mandrilhadoras e acessórios, além de prestar serviços técnicos especializados. A competitividade se acirra com fornecedores que oferecem soluções completas e atendimento regionalizado.\n\n---\n\n## 3. Avaliação Competitiva\n\n### 3.1 Grupo Franzen\n\n**Localização:** MG, SP e PA  \n**Atuação:** Diversas máquina

In [8]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Document types found: {', '.join(doc_types)}")

Document types found: company, products


In [9]:
for chunk in chunks:
    if 'jvf' in chunk.page_content:
        print(chunk)
        print("_________")

page_content='**Diferenciais:**

- Representação exclusiva no Brasil
- Equipamentos de alta eficiência e suporte técnico nacional
- Tecnologia com preço competitivo
- Compromisso ambiental com soluções sustentáveis

**Contato:**

- WhatsApp: +55 (47) 99132-9067
- E-mail: contato@jvfmaquinas.com.br
- Endereço: Rua Pouso Alegre, 149 – Joinville, SC
- Site: https://jvfmaquinas.com.br

---

## 2. Catálogo Institucional

### Sobre a JVF

- Empresa com sede em Joinville–SC, fundada por profissionais de MG
- Mais de 20 anos de experiência no setor
- Mais de 10 anos de atuação em importação
- Equipe técnica qualificada com presença nacional
- Parceria sólida com a fábrica Y S Power

### Sobre a Y S Power

- Fabricante global reconhecida em solda e usinagem portátil
- Equipamentos de alto desempenho e confiabilidade
- Suporte técnico mundial com treinamentos online e presenciais
- Excelência em controle de qualidade

---

## 3. Manual Técnico – Mandrilhadora Portátil S50' metadata={'source': '.